In [5]:
%help


Available Magic Commands

## Sessions Magics
%help | Return a list of descriptions and input types for all magic commands. 
%profile | String | Specify a profile in your aws configuration to use as the credentials provider.
%region | String | Specify the AWS region in which to initialize a session | Default from ~/.aws/configure
%idle_timeout | Int | The number of minutes of inactivity after which a session will timeout. The default idle timeout value is 2880 minutes (48 hours).
%session_id | Returns the session ID for the running session. 
%session_id_prefix | String | Define a String that will precede all session IDs in the format [session_id_prefix]-[session_id]. If a session ID is not provided, a random UUID will be generated.
%status | Returns the status of the current Glue session including its duration, configuration and executing user / role.
%list_sessions | Lists all currently running sessions by name and ID.
%stop_session | Stops the current session.
%glue_version | String 

In [6]:
%session_id_prefix delta-analysis
%glue_version 3.0
%idle_timeout 60
%%configure
{
"--enable-spark-ui": "true",
"--spark-event-logs-path": "s3://gis-spark-logs/",
"--spark.jars.packages": "io.delta:delta-core_2.12:1.0.1",
"--spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
"--spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.35 and you have 0.34 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Setting session ID prefix to delta-analysis
Setting Glue version to: 3.0
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 60 minutes.
The following configurations have been updated: {'--enable-spark-ui': 'true', '--spark-event-logs-path': 's3://gis-spark-logs/', '--spark.jars.packages': 'io.delta:delta-core_2.12:1.0.1', '--spark.sql.extensions': 'io.delta.sql.DeltaSparkSessionExtension', '--spark.sql.catalog.spark_catalog': 'org.apache.spark.sql.delta.catalog.Delt

In [ ]:
from pyspark.sql.functions import split, lower, hour
print(spark.version)
day_to_analyze = "2022-01-05"
df = spark.read.json(f"s3://openaq-fetches/realtime-gzipped/{day_to_analyze}/1641409725.ndjson.gz")
df_air = spark.read.schema(df.schema).json(f"s3://openaq-fetches/realtime-gzipped/{day_to_analyze}/*")

Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: air-analysis--76fdac11-3ae9-4fbc-ac5b-9cbcec73c7e8
Applying the following default arguments:
--glue_kernel_version 0.34
--enable-glue-datacatalog true
--enable-spark-ui true
--spark-event-logs-path s3://<BUCKET>/gis-spark-logs/
--spark.jars.packages io.delta:delta-core_2.12:2.0.0
--spark.sql.extensions io.delta.sql.DeltaSparkSessionExtension
--spark.sql.catalog.spark_catalog org.apache.spark.sql.delta.catalog.DeltaCatalog
Waiting for session air-analysis--76fdac11-3ae9-4fbc-ac5b-9cbcec73c7e8 to get into ready status...
Session air-analysis--76fdac11-3ae9-4fbc-ac5b-9cbcec73c7e8 has been created

3.1.1-amzn-0


In [1]:
df_city = df_air.filter(lower((df_air.city)).contains('delhi')).filter(df_air.parameter == "no2").cache()
df_avg = df_city.withColumn("Hour", hour(df_city.date.utc)).groupBy("Hour").avg("value").withColumnRenamed("avg(value)", "no2_avg")
df_avg.sort("Hour").show()

+----+------------------+
|Hour|           no2_avg|
+----+------------------+
|   0|         37.823125|
|   1|40.444137931034476|
|   2|            45.133|
|   3|         37.654375|
|   4|53.437500000000014|
|   5|           58.0984|
|   6|58.881666666666675|
|   7| 78.51407407407407|
|   8| 65.86359999999999|
|   9| 43.94166666666667|
|  10|47.756190476190476|
|  11|38.246562499999996|
|  12| 45.30607142857143|
|  13|42.872413793103455|
|  14| 45.60785714285715|
|  15| 38.22392857142857|
|  16|32.498888888888885|
|  17| 31.73413793103448|
|  18| 32.16227272727272|
|  19|28.679062499999997|
+----+------------------+
only showing top 20 rows


In [8]:
from pyspark.sql.functions import split, lower, hour
print(spark.version)

3.1.1-amzn-0
